In [48]:
from glob import glob
from collections import Counter
import keras
from keras.datasets import mnist
from keras.layers import Dense, Dropout
from keras.models import load_model, Sequential
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
def prepare_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype(np.float32)/255.0
    x_test = x_test.astype(np.float32)/255.0
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    return x_train, y_train, x_test, y_test

In [43]:
def create_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=SGD(),
                  metrics=['accuracy'])
    return model

In [50]:
histories = list()
models = list()
for i in range(7):
    print(f'model {i + 1}')
    model = create_model()
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)
    history = model.fit(x_train, y_train, batch_size=64, epochs=10,
                        validation_data=(x_val, y_val), verbose=0)
    models.append(model)
    histories.append(history)

Train on 45000 samples, validate on 15000 samples
Epoch 1/10
45000/45000 [==============================] - 136s 3ms/step - loss: 1.0269 - acc: 0.7287 - val_loss: 0.4541 - val_acc: 0.8817
Epoch 2/10
45000/45000 [==============================] - 96s 2ms/step - loss: 0.4596 - acc: 0.8672 - val_loss: 0.3340 - val_acc: 0.9074
Epoch 3/10
45000/45000 [==============================] - 49s 1ms/step - loss: 0.3775 - acc: 0.8903 - val_loss: 0.2894 - val_acc: 0.9181
Epoch 4/10
45000/45000 [==============================] - 53s 1ms/step - loss: 0.3329 - acc: 0.9040 - val_loss: 0.2595 - val_acc: 0.9249
Epoch 5/10
45000/45000 [==============================] - 53s 1ms/step - loss: 0.3021 - acc: 0.9118 - val_loss: 0.2405 - val_acc: 0.9309
Epoch 6/10
45000/45000 [==============================] - 49s 1ms/step - loss: 0.2761 - acc: 0.9198 - val_loss: 0.2189 - val_acc: 0.9361
Epoch 7/10
45000/45000 [==============================] - 155s 3ms/step - loss: 0.2548 - acc: 0.9256 - val_loss: 0.2164 - val_a

KeyboardInterrupt: 

In [11]:
models = list()
for model_file in glob("Models/mnist_mpl_0*.h5"):
    print(model_file)
    models.append(load_model(model_file))

Models/mnist_mpl_04.h5
Models/mnist_mpl_09.h5
Models/mnist_mpl_03.h5
Models/mnist_mpl_02.h5
Models/mnist_mpl_01.h5
Models/mnist_mpl_06.h5
Models/mnist_mpl_08.h5
Models/mnist_mpl_07.h5
Models/mnist_mpl_05.h5


In [22]:
accuracies = list()
for model in models:
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    accuracies.append(accuracy)

In [23]:
print(min(accuracies), max(accuracies))

0.9763 0.9774


In [29]:
y = models[0].predict_classes(x_test)

In [31]:
y.dtype

dtype('int64')

In [33]:
y_infer = np.empty((len(models), len(y_test)), dtype=np.int64)

In [34]:
for i, model in enumerate(models):
    y_infer[i, :] = model.predict_classes(x_test)

In [35]:
y_infer.shape

(9, 10000)

In [41]:
mistakes = 0
for j in range(y_infer.shape[1]):
    counter = Counter()
    for i in range(y_infer.shape[0]):
        counter[y_infer[i, j]] += 1
    inferred = counter.most_common(1)[0][0]
    if inferred != np.argmax(y_test[j]):
        mistakes += 1
print(1.0 - mistakes/len(y_test))

0.9781
